In [1]:
from algo_trading.market_data import MarketData
from algo_trading.backtest.backtester import BackTester
from algo_trading.backtest.trading_algorithm import TradingAlgorithm

In [2]:
%time market_data = MarketData('./stock_data.csv')

CPU times: user 6.02 s, sys: 503 ms, total: 6.52 s
Wall time: 5.87 s


### 삼성전자(005930) 종목에 대해 골든/데스크로스 전략 적용
### 2016-01-01 ~ 2019-06-30 기간에 대해 테스트
### 초기자본금 10,000,000원으로 설정

In [3]:
bt = BackTester(market_data, initial_cash=1e7, start_date='2016-01-01', end_date='2019-06-30')

In [4]:
class GoldenDeathCross(TradingAlgorithm):

    def make_order(self, order, date, market_data, context):

        symbol = '005930' # 삼성전자

        long_term_data = market_data.get_recent(symbol, date, n_days=20)
        short_term_data = market_data.get_recent(symbol, date, n_days=5)

        long_term_ma = long_term_data.Close.mean()
        short_term_ma = short_term_data.Close.mean()

        # 단기(최근 5일간) 가격 평균이 장기(최근 20일간) 가격 평균보다 높으면 매수
        if short_term_ma > long_term_ma:
            order.add(symbol, 10)

        # 반대면 매도
        elif short_term_ma < long_term_ma:
            order.add(symbol, -10)

        return order

In [5]:
gdc = GoldenDeathCross()

In [6]:
%time result = bt.run(gdc)

100%|██████████| 854/854 [00:02<00:00, 333.20it/s]

CPU times: user 2.52 s, sys: 68 ms, total: 2.59 s
Wall time: 2.58 s


In [7]:
result

{'return': 0.20335405, 'adjusted_return': 0.09293899027392698}

### 20% 수익률 기록. 동일 기간 코스피 수익률 대비 9.3%p 높음